<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=a77d186ba91a56a2c946f074365f1ff75305eff76c37e63c4293107cc3151051
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-25 09:33:26
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -52.11 K (-0.37%)
Current PnL: -23.86 L (-15.63%)
CY Booked + Current PnL: -9.97 L (-6.53%)
-------------------
Total profit:  1.29 L
Total loss:  -25.15 L
-------------------
Total Booked + Current PnL: 17.01 L (13.54%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.79%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 89.27 L (62.9%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 6.94%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.02,-14.20,16.66,0.09,14405.0,-14310.0,86467.0,100.64,49.0,M-SC,3.97,253.0,-0.99,0.61,13.60,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.20,-9.15,10.09,0.02,16001.0,-15975.0,158584.0,141.63,70.0,M-SC,6.68,234.0,-1.00,1.12,37.94,OX40N,NTT,PAINTS
35,ICICIGI,2252.93,0.38,6.97,12.37,20.19,23066.0,12145.0,186465.0,-16.25,51.0,X-MC,7.59,68.0,0.53,1.31,23.13,X40,ATH,INSURANCE
43,ITC,452.00,-0.27,-1.67,12.31,10.44,24226.0,-3340.0,196798.0,-41.15,41.0,X-LC,0.83,5.0,-0.14,1.39,3.80,X40,NTT,FMCG
51,MASFIN,398.61,0.29,-4.93,28.38,22.05,26436.0,-4830.0,93150.0,-18.25,51.0,H-SC,7.35,164.0,-0.18,0.66,35.40,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,ROUTE,2227.21,3.39,-49.62,235.55,69.05,157910.0,-66027.0,67039.0,-59.29,36.0,H-SC,23.57,140.0,-0.42,0.47,3.39,SR,ATH,IT
18,COFFEEDAY,80.00,2.65,-37.86,117.69,35.27,83042.0,-42989.0,70560.0,-51.71,37.0,L-SC,17.80,270.0,-0.52,0.50,72.70,XR,NTT,HOTELS
56,QUESS,424.00,1.53,-28.04,98.50,42.84,46076.0,-18228.0,46778.0,-52.62,40.0,X-SC,37.38,83.0,-0.40,0.33,1.79,XY24,NTT,MISC
61,REPCOHOME,880.00,1.15,-11.99,115.69,89.82,291705.0,-34361.0,252144.0,-56.04,46.0,H-SC,10.35,160.0,-0.12,1.78,33.40,XY24,NTT,FINANCE
58,RAJOOENG,143.10,0.92,-30.15,99.86,39.61,71500.0,-30900.0,71600.0,-52.76,22.0,H-SC,20.08,135.0,-0.43,0.50,0.92,AR,ATH,MISC


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
8,AWL,485.00,-3.51,-15.23,81.07,53.50,207375.0,-45954.0,255797.0,-58.91,45.0,X-MC,4.15,58.0,-0.22,1.80,13.62,XY24,NTT,FMCG
34,HONAUT,58357.33,-2.23,-12.93,64.60,43.32,91616.0,-21052.0,141820.0,-25.23,47.0,X-SC,1.95,90.0,-0.23,1.00,8.89,X40N,ATH,ELECTRICAL
3,ACC,3906.00,-2.10,-21.27,108.49,64.15,203256.0,-50601.0,187350.0,-54.13,55.0,X-SC,1.62,82.0,-0.25,1.32,5.09,XY24,BTT,CEMENT
57,RAJESHEXPO,518.00,-2.08,-63.79,176.34,0.07,87595.0,-87503.0,49674.0,1720.71,54.0,L-SC,1.53,268.0,-1.00,0.35,23.54,OX40N,NTT,JEWELLERY
81,VALIANTORG,838.00,-2.04,-48.05,235.20,74.15,199920.0,-78605.0,85000.0,-252.84,22.0,H-SC,15.59,149.0,-0.39,0.60,8.96,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.77,0.64,71.17,72.27,121368.0,1084.0,170532.0,-13.79,53.0,M-LC,4.55,99.0,0.01,1.2,6.85,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.18,-5.45,119.90,107.91,167376.0,-8048.0,139596.0,-22.98,32.0,M-SC,10.49,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.20,-9.15,10.09,0.02,16001.0,-15975.0,158584.0,141.63,70.0,M-SC,6.68,234.0,-1.00,1.12,37.94,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.02,-14.20,16.66,0.09,14405.0,-14310.0,86467.0,100.64,49.0,M-SC,3.97,253.0,-0.99,0.61,13.60,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.80,-23.19,47.74,13.47,98882.0,-62541.0,207126.0,-68.51,25.0,H-SC,1.38,158.0,-0.63,1.46,5.32,XY24,NTT,PAINTS
44,JCHAC,2282.00,-0.43,-23.53,30.76,-0.01,30062.0,-30074.0,97731.0,19900.00,53.0,M-SC,1.67,233.0,-1.00,0.69,10.67,OX40N,NTT,AC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.77,0.64,71.17,72.27,121368.0,1084.0,170532.0,-13.79,53.0,M-LC,4.55,99.0,0.01,1.2,6.85,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.77,0.64,71.17,72.27,121368.0,1084.0,170532.0,-13.79,53.0,M-LC,4.55,99.0,0.01,1.20,6.85,XR,NTT,IT
38,INDIAMART,4810.62,-0.61,-1.44,109.75,106.72,133408.0,-1780.0,121556.0,-51.20,31.0,H-SC,3.66,139.0,-0.01,0.86,21.60,AR,ATH,MISC
25,FINCABLES,1641.55,-0.18,-5.45,119.90,107.91,167376.0,-8048.0,139596.0,-22.98,32.0,M-SC,10.49,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,-0.10,-2.38,38.93,35.63,79647.0,-4983.0,204591.0,-16.30,33.0,H-MC,4.37,119.0,-0.06,1.44,13.90,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-0.85,-5.99,93.50,81.92,100910.0,-6873.0,107925.0,-44.94,31.0,M-SC,4.48,236.0,-0.07,0.76,27.77,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.84,-15.81,68.87,42.18,160276.0,-43688.0,232722.0,-24.30,21.0,X-LC,0.61,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
46,JSWINFRA,342.0,-0.63,-9.94,27.95,15.23,50430.0,-19919.0,180428.0,-22.98,23.0,X-MC,6.66,66.0,-0.39,1.27,19.10,X40N,NTT,REALTY
11,BATAINDIA,2096.0,-0.46,-35.42,109.37,35.20,90877.0,-45579.0,83091.0,8.88,24.0,X-SC,12.89,92.0,-0.50,0.59,0.07,X40,NTT,FOOTWEAR
60,RELAXO,1176.0,-0.41,-48.03,193.08,52.31,145650.0,-69725.0,75435.0,-44.63,26.0,X-SC,3.97,91.0,-0.48,0.53,0.97,X40N,NTT,FOOTWEAR
27,GLAXO,3466.2,-1.21,-1.77,41.65,39.15,81534.0,-3524.0,195760.0,-21.11,29.0,X-MC,9.47,60.0,-0.04,1.38,26.72,X40,ATH,PHARMA


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.0,-0.84,-15.81,68.87,42.18,160276.0,-43688.0,232722.0,-24.30,21.0,X-LC,0.61,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
43,ITC,452.0,-0.27,-1.67,12.31,10.44,24226.0,-3340.0,196798.0,-41.15,41.0,X-LC,0.83,5.0,-0.14,1.39,3.80,X40,NTT,FMCG
36,ICICIPRULI,790.0,0.44,1.13,29.51,30.97,52743.0,1992.0,178730.0,-22.64,48.0,X-MC,1.57,75.0,0.04,1.26,13.83,X40,ATH,INSURANCE
3,ACC,3906.0,-2.10,-21.27,108.49,64.15,203256.0,-50601.0,187350.0,-54.13,55.0,X-SC,1.62,82.0,-0.25,1.32,5.09,XY24,BTT,CEMENT
66,SIEMENS,4671.5,0.60,-14.48,46.77,25.51,74432.0,-26950.0,159145.0,-2.24,55.0,H-LC,1.88,49.0,-0.36,1.12,17.28,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-0.49,-13.92,72.46,48.45,125062.0,-27912.0,172594.0,-20.09,29.0,X-MC,6.83,64.0,-0.22,1.22,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.07,-0.86,-5.89,33.95,26.07,52317.0,-9640.0,154100.0,-30.68,33.0,X-MC,9.05,55.0,-0.18,1.09,0.00,X40,ATH,APPARELS
76,TMPV,600.00,-0.84,-15.81,68.87,42.18,160276.0,-43688.0,232722.0,-24.30,21.0,X-LC,0.61,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.46,-35.42,109.37,35.20,90877.0,-45579.0,83091.0,8.88,24.0,X-SC,12.89,92.0,-0.50,0.59,0.07,X40,NTT,FOOTWEAR
55,PGHH,17907.65,-0.48,-4.59,40.19,33.76,77008.0,-9210.0,191610.0,-32.64,33.0,X-MC,3.37,57.0,-0.12,1.35,0.37,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,1.53,-28.04,98.50,42.84,46076.0,-18228.0,46778.0,-52.62,40.0,X-SC,37.38,83.0,-0.40,0.33,1.79,XY24,NTT,MISC
60,RELAXO,1176.00,-0.41,-48.03,193.08,52.31,145650.0,-69725.0,75435.0,-44.63,26.0,X-SC,3.97,91.0,-0.48,0.53,0.97,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.46,-35.42,109.37,35.20,90877.0,-45579.0,83091.0,8.88,24.0,X-SC,12.89,92.0,-0.50,0.59,0.07,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,-1.78,2.55,20.99,24.08,27327.0,3239.0,130189.0,-5.81,44.0,X-SC,4.97,89.0,0.12,0.92,22.67,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,0.38,-5.43,28.57,21.58,37862.0,-7614.0,132525.0,-52.33,33.0,X-MC,6.82,56.0,-0.20,0.93,17.33,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-0.05,-14.70,39.83,19.28,117544.0,-50843.0,295113.0,-25.57,61.0,X-LC,7.52,1.0,-0.43,2.08,9.10,X40,ATH,IT
41,INFY,2275.00,-0.59,4.76,47.83,54.87,156780.0,14898.0,327786.0,-18.52,58.0,X-LC,3.00,2.0,0.10,2.31,13.44,X40,BTT,IT
76,TMPV,600.00,-0.84,-15.81,68.87,42.18,160276.0,-43688.0,232722.0,-24.30,21.0,X-LC,0.61,3.0,-0.27,1.64,0.00,XY24,NTT,AUTO
82,VBL,671.64,0.29,-9.63,50.12,35.67,143063.0,-30401.0,285441.0,-20.30,41.0,X-LC,5.95,4.0,-0.21,2.01,2.90,X40N,ATH,FMCG
43,ITC,452.00,-0.27,-1.67,12.31,10.44,24226.0,-3340.0,196798.0,-41.15,41.0,X-LC,0.83,5.0,-0.14,1.39,3.80,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,-0.38,-13.57,110.19,81.67,89346.0,-12726.0,81084.0,7288.89,49.0,L-SC,18.71,271.0,-0.14,0.57,57.02,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.90,-13.29,60.92,39.54,53438.0,-13440.0,87719.0,-32.23,63.0,M-SC,6.70,220.0,-0.25,0.62,23.69,AR,ATH,AUTO
51,MASFIN,398.61,0.29,-4.93,28.38,22.05,26436.0,-4830.0,93150.0,-18.25,51.0,H-SC,7.35,164.0,-0.18,0.66,35.40,XR,ATH,FINANCE
70,SURYODAY,216.00,0.82,-19.17,52.21,23.03,75573.0,-34323.0,144748.0,57.89,49.0,H-SC,10.64,167.0,-0.45,1.02,43.62,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.20,-9.15,10.09,0.02,16001.0,-15975.0,158584.0,141.63,70.0,M-SC,6.68,234.0,-1.00,1.12,37.94,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.20,-9.15,10.09,0.02,16001.0,-15975.0,158584.0,141.63,70.0,M-SC,6.68,234.0,-1.00,1.12,37.94,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.17,0.80,19.98,20.95,50708.0,2024.0,253792.0,-0.21,77.0,X-LC,13.39,31.0,0.04,1.79,36.73,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,-1.24,-38.28,115.30,32.89,54946.0,-29552.0,47655.0,-688.19,59.0,L-MC,5.57,259.0,-0.54,0.34,31.24,XR,NTT,BANKS
12,BERGEPAINT,680.00,0.22,3.42,17.13,21.14,40277.0,7784.0,235123.0,-3.83,65.0,X-MC,3.93,74.0,0.19,1.66,32.53,XY24,NTT,PAINTS
79,UNITDSPR,1644.00,-0.42,4.69,15.35,20.77,37628.0,10990.0,245134.0,-4.44,57.0,X-MC,3.55,62.0,0.29,1.73,11.94,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.75
1,25,44.66
2,50,75.53


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.62
MC    29.26
LC    25.15
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.92
X40      21.24
X40N     11.91
XR        9.64
XY25      9.06
AR        8.96
OX40N     7.67
X200      1.87
SR        0.99
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.90
X-MC    22.83
X-LC    19.99
M-SC    12.02
X-SC     8.28
H-MC     4.72
H-LC     2.99
L-SC     1.42
M-MC     1.37
M-LC     1.20
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.75,-5.91,40.16
IT,13.78,-15.54,75.18
FINANCE,9.44,-15.67,64.54
MISC,7.42,-23.57,74.19
PAINTS,6.03,-8.04,24.09
ELECTRICAL,5.93,-11.47,52.30
INSURANCE,4.44,-0.35,35.71
PHARMA,3.86,-2.03,34.68
AUTO,3.34,-21.45,73.53


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3075987.0,21
AR,1299956.0,10
XR,1295035.0,13
X40,1026376.0,14
X40N,758657.0,9
OX40N,702331.0,10
XY25,355583.0,6
SR,279464.0,2
MH,74509.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3585844.0,25
M-SC,1353715.0,15
X-MC,1243757.0,16
X-LC,927294.0,11
X-SC,745963.0,8
H-MC,408793.0,3
L-SC,259983.0,3
H-LC,133416.0,2
M-LC,121368.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1225038.0      6
           AR         883057.0      5
           XR         795701.0      7
M-SC       XY24       791785.0      6
X-MC       X40        451521.0      7
X-LC       X40        398880.0      5
X-MC       XY24       356969.0      3
H-SC       OX40N      328075.0      4
X-SC       X40N       293329.0      3
M-SC       OX40N      286661.0      5
H-SC       SR         279464.0      2
X-SC       XY24       276659.0      3
X-MC       X40N       250982.0      4
X-LC       XY24       234573.0      2
H-MC       AR         217830.0      2
X-LC       X40N       214346.0      2
H-MC       XY24       190963.0      1
X-MC       XY25       184285.0      2
X-SC       X40        175975.0      2
L-SC       XR         172388.0      2
M-SC       XR         150632.0      2
           AR         124637.0      2
M-LC       XR         121368.0      1
L-SC       OX40N       87595.0      1
X-LC       XY25        79495.0      2
H-SC       MH          74509.0      1
H-LC       AR          74432.0      1
           X200        58984.0      1
L-MC       XR          54946.0      1
M-MC       XY25        50183.0      1
L-LC       XY25        41620.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 26.0 seconds
